In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer

import sys
sys.path.append("F:\Programming\Projects\GlassDoor sentiment analysis\Phase 1\Src")
import data_preprocessing


%matplotlib inline

In [2]:
data = pd.read_csv('../../Data/glassdoor_reviews.csv')
data.head()

,firm,date_review,job_title,current,location,overall_rating,work_life_balance,culture_values,diversity_inclusion,career_opp,comp_benefits,senior_mgmt,recommend,ceo_approv,outlook,headline,pros,cons
0,AFH-Wealth-Management,2015-04-05,,Current Employee,NaN,2,4.0,3.0,NaN,2.0,3.0,3.0,x,o,r,"Young colleagues, poor micro management",Very friendly and welcoming to new staff. Easy...,"Poor salaries, poor training and communication."
1,AFH-Wealth-Management,2015-12-11,Office Administrator,"Current Employee, more than 1 year","Bromsgrove, England, England",2,3.0,1.0,NaN,2.0,1.0,4.0,x,o,r,"Excellent staff, poor salary","Friendly, helpful and hard-working colleagues",Poor salary which doesn't improve much with pr...
2,AFH-Wealth-Management,2016-01-28,Office Administrator,"Current Employee, less than 1 year","Bromsgrove, England, England",1,1.0,1.0,NaN,1.0,1.0,1.0,x,o,x,"Low salary, bad micromanagement",Easy to get the job even without experience in...,"Very low salary, poor working conditions, very..."
3,AFH-Wealth-Management,2016-04-16,,Current Employee,NaN,5,2.0,3.0,NaN,2.0,2.0,3.0,x,o,r,Over promised under delivered,Nice staff to work with,No career progression and salary is poor
4,AFH-Wealth-Management,2016-04-23,Office Administrator,"Current Employee, more than 1 year","Bromsgrove, England, England",1,2.0,1.0,NaN,2.0,1.0,1.0,x,o,x,client reporting admin,"Easy to get the job, Nice colleagues.","Abysmal pay, around minimum wage. No actual tr..."


In [3]:
data.columns

Index(['firm', 'date_review', 'job_title', 'current', 'location',
       'overall_rating', 'work_life_balance', 'culture_values',
       'diversity_inclusion', 'career_opp', 'comp_benefits', 'senior_mgmt',
       'recommend', 'ceo_approv', 'outlook', 'headline', 'pros', 'cons'],
      dtype='object')

In [4]:
len(data)

838566

In [3]:
data.isnull().sum()/len(data)*100

firm                    0.000000
date_review             0.000000
job_title               0.000000
current                 0.000000
location               35.458509
overall_rating          0.000000
work_life_balance      17.875039
culture_values         22.821459
diversity_inclusion    83.773967
career_opp             17.589671
comp_benefits          17.897458
senior_mgmt            18.588400
recommend               0.000000
ceo_approv              0.000000
outlook                 0.000000
headline                0.308861
pros                    0.000239
cons                    0.001550
dtype: float64

# Data Cleaning

## Unnecessary columns removal

### diversity_inclusion column has so many missing values around 84%, and it's not necessary for our model or analysis so better to remove it

In [3]:
data_cleaned = data.copy()

data_cleaned.drop('diversity_inclusion', inplace = True, axis = 1)
data_cleaned.isnull().sum()/len(data_cleaned)*100

firm                  0.000000
date_review           0.000000
job_title             0.000000
current               0.000000
location             35.458509
overall_rating        0.000000
work_life_balance    17.875039
culture_values       22.821459
career_opp           17.589671
comp_benefits        17.897458
senior_mgmt          18.588400
recommend             0.000000
ceo_approv            0.000000
outlook               0.000000
headline              0.308861
pros                  0.000239
cons                  0.001550
dtype: float64

## Missing values Imputing

### For the rest of the numeric columns we can impute them

In [7]:
imputer = SimpleImputer(strategy='median')

In [8]:
missing_numeric_cols = ['work_life_balance', 'culture_values', 'career_opp', 'comp_benefits', 'senior_mgmt']

data_cleaned[missing_numeric_cols] = pd.DataFrame(imputer.fit_transform(data_cleaned[missing_numeric_cols]), columns = missing_numeric_cols)

In [5]:
data_cleaned.isnull().sum()/len(data_cleaned)*100

firm                  0.000000
date_review           0.000000
job_title             0.000000
location             35.458509
overall_rating        0.000000
work_life_balance    17.875039
culture_values       22.821459
career_opp           17.589671
comp_benefits        17.897458
senior_mgmt          18.588400
recommend             0.000000
ceo_approv            0.000000
outlook               0.000000
headline              0.308861
pros                  0.000239
cons                  0.001550
current_state         0.000000
emp_state             0.000000
emp_working_years     0.000000
dtype: float64

## Text columns cleaning

### Headline, Pros, and Cons columns

For text columns like headline, pros, and cons, we can replace the missing values with space or empty string

In [10]:
data_cleaned[['headline', 'pros', 'cons']] = data_cleaned[['headline', 'pros', 'cons']].fillna('')

In [11]:
data_cleaned.isnull().sum()/len(data_cleaned)*100

firm                  0.000000
date_review           0.000000
job_title             0.000000
current               0.000000
location             35.458509
overall_rating        0.000000
work_life_balance     0.000000
culture_values        0.000000
career_opp            0.000000
comp_benefits         0.000000
senior_mgmt           0.000000
recommend             0.000000
ceo_approv            0.000000
outlook               0.000000
headline              0.000000
pros                  0.000000
cons                  0.000000
dtype: float64

### Location Column

First we should simply fill the missing values with "Unknown"

In [8]:
data_cleaned['location'] = data_cleaned['location'].fillna('Unknown')

### Current Column

Let's split this column into three meaningfull columns:
- current: "Current", "Former"
- emp state: "Employee", "Temporary Employee", "Contractor", "Intern", Freelancer
- emp working years

In [13]:
data_cleaned['current'].unique()

array(['Current Employee', 'Current Employee, more than 1 year',
       'Current Employee, less than 1 year', 'Former Employee',
       'Current Employee, more than 5 years',
       'Former Employee, more than 1 year',
       'Former Employee, more than 3 years',
       'Former Employee, more than 5 years',
       'Current Employee, more than 3 years',
       'Current Employee, more than 8 years',
       'Former Employee, less than 1 year',
       'Former Employee, more than 8 years',
       'Current Employee, more than 10 years',
       'Former Employee, more than 10 years',
       'Former Contractor, less than 1 year',
       'Former Intern, less than 1 year',
       'Current Contractor, less than 1 year', 'Former Contractor',
       'Former Intern, more than 1 year', 'Current Contractor',
       'Former Intern', 'Current Intern, less than 1 year',
       'Current Contractor, more than 1 year',
       'Former Contractor, more than 1 year',
       'Former Contractor, more than 8 years

In [4]:
data_cleaned = data_preprocessing.parse_job_status(data_cleaned, column = "current")
data_cleaned = data_cleaned.drop('current', axis = 1)
data_cleaned.head()

,firm,date_review,job_title,location,overall_rating,work_life_balance,culture_values,career_opp,comp_benefits,senior_mgmt,recommend,ceo_approv,outlook,headline,pros,cons,current_state,emp_state,emp_working_years
0,AFH-Wealth-Management,2015-04-05,,NaN,2,4.0,3.0,2.0,3.0,3.0,x,o,r,"Young colleagues, poor micro management",Very friendly and welcoming to new staff. Easy...,"Poor salaries, poor training and communication.",Current,Employee,Unknown
1,AFH-Wealth-Management,2015-12-11,Office Administrator,"Bromsgrove, England, England",2,3.0,1.0,2.0,1.0,4.0,x,o,r,"Excellent staff, poor salary","Friendly, helpful and hard-working colleagues",Poor salary which doesn't improve much with pr...,Current,Employee,more than 1 year
2,AFH-Wealth-Management,2016-01-28,Office Administrator,"Bromsgrove, England, England",1,1.0,1.0,1.0,1.0,1.0,x,o,x,"Low salary, bad micromanagement",Easy to get the job even without experience in...,"Very low salary, poor working conditions, very...",Current,Employee,less than 1 year
3,AFH-Wealth-Management,2016-04-16,,NaN,5,2.0,3.0,2.0,2.0,3.0,x,o,r,Over promised under delivered,Nice staff to work with,No career progression and salary is poor,Current,Employee,Unknown
4,AFH-Wealth-Management,2016-04-23,Office Administrator,"Bromsgrove, England, England",1,2.0,1.0,2.0,1.0,1.0,x,o,x,client reporting admin,"Easy to get the job, Nice colleagues.","Abysmal pay, around minimum wage. No actual tr...",Current,Employee,more than 1 year


In [ ]:
# cols_to_add = ['current_state', 'emp_state', 'emp_working_years']
# df_temp = pd.read_csv('../../Data/Analysis_data.csv')
# df_temp = pd.concat([df_temp, data_cleaned[cols_to_add]], axis=1)
# df_temp.to_csv('../../Data/Analysis_data_.csv')

### For the text survey columns, I'll map the values for better interpretting

In [13]:
data_cleaned = data_preprocessing.preprocess_survey_columns(data_cleaned)
data_cleaned = data_cleaned.drop(['recommend', 'ceo_approv', 'outlook'], axis= 1)
data_cleaned.head()

,firm,date_review,job_title,current,location,overall_rating,work_life_balance,culture_values,career_opp,comp_benefits,senior_mgmt,headline,pros,cons,outlook_clean,recommend_clean,ceo_approv_clean,sentiment
0,AFH-Wealth-Management,2015-04-05,,Current Employee,Unknown,2,4.0,3.0,2.0,3.0,3.0,"Young colleagues, poor micro management",Very friendly and welcoming to new staff. Easy...,"Poor salaries, poor training and communication.",Negative,No,Disapprove,Negative
1,AFH-Wealth-Management,2015-12-11,Office Administrator,"Current Employee, more than 1 year","Bromsgrove, England, England",2,3.0,1.0,2.0,1.0,4.0,"Excellent staff, poor salary","Friendly, helpful and hard-working colleagues",Poor salary which doesn't improve much with pr...,Negative,No,Disapprove,Negative
2,AFH-Wealth-Management,2016-01-28,Office Administrator,"Current Employee, less than 1 year","Bromsgrove, England, England",1,1.0,1.0,1.0,1.0,1.0,"Low salary, bad micromanagement",Easy to get the job even without experience in...,"Very low salary, poor working conditions, very...",Other,No,Disapprove,Negative
3,AFH-Wealth-Management,2016-04-16,,Current Employee,Unknown,5,2.0,3.0,2.0,2.0,3.0,Over promised under delivered,Nice staff to work with,No career progression and salary is poor,Negative,No,Disapprove,Positive
4,AFH-Wealth-Management,2016-04-23,Office Administrator,"Current Employee, more than 1 year","Bromsgrove, England, England",1,2.0,1.0,2.0,1.0,1.0,client reporting admin,"Easy to get the job, Nice colleagues.","Abysmal pay, around minimum wage. No actual tr...",Other,No,Disapprove,Negative


# Now this data is Ready for the analysis, building a useful Dashboard, and building a model

In [14]:
# analysis_file_name = 'Analysis_data.csv'
sentiment_file_name = 'Sentiment_data_2.csv'

sentiment_cols = ['headline', 'pros', 'cons', 'sentiment']

# data_cleaned.drop(sentiment_cols, axis=1).to_csv(f'../Data/{analysis_file_name}', index=False)

data_cleaned[sentiment_cols].to_csv(f'../Data/{sentiment_file_name}', index=False)
